## Evaluation of fine-tuned BERT base uncased models on validation datasets
### this notebook contains functions and fragments of code taken from the Jypyter notebook available on HuggingFace GitHub https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb

In [1]:
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import pandas as pd
from datasets import ClassLabel, Sequence
from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import default_data_collator
from datasets import Dataset
import collections
import numpy as np
from tqdm.auto import tqdm

/home/luki/anaconda3/envs/env-01/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# parameter for metrics
squad_v2 = False

# parameter for model
model_checkpoint = "bert-base-uncased"

# parameter for batch size
batch_size = 16

In [3]:
# name for the model
m_name = 'squad_base'

# path to the saved BERT model
saved_model = 'models/saved_baseline_model'

In [4]:
# loading of tokenizer from saved model
tokenizer = AutoTokenizer.from_pretrained(saved_model)
# loading of saved fine-tuned model
model = AutoModelForQuestionAnswering.from_pretrained(saved_model)
# data collator
data_collator = default_data_collator
#loading of metrics for evaluation
metric = load_metric("squad_v2" if squad_v2 else "squad")

In [5]:
# parameters from the QA HuggingFace Jupyter Notebook
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [6]:
# training arguments for the saved model
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad_with_callbacks",
    evaluation_strategy = "steps",
    eval_steps = 200,
    save_steps = 200,
    logging_steps = 200,
    save_total_limit = 5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
)

In [7]:
# Trainer for model evaluation
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

In [8]:
# Preprocessing function for train dataset from the HuggingFace Jupyter notebook
# with original comments
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [9]:
# Preprocessing function for validation dataset from the HuggingFace Jupyter notebook
# with original comments
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [10]:
# Postprocessing function from the HuggingFace Jupyter notebook
# with original comments
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    if len(offset_mapping[start_index]) == 0 or len(offset_mapping[end_index]) == 0: 
                        continue
                                        
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [11]:
def model_evaluation_on_dataset(dataset, save_dataframe_with_predictions = False, name = 'model_name'):
    """Model evaluation on specific dataset
    Calls previous functions and evaluate the dataset on the model for exact match and F1

    Args:
        dataset (Dataset): validation dataset
        save_dataframe_with_predictions (bool, optional): flag for saving the dataset with prediction. Defaults to False.
        name (str, optional): name of the model. Defaults to 'model_name'.

    Returns:
        _type_: _description_
    """
    validation_features = dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=dataset.column_names
    )

    raw_predictions = trainer.predict(validation_features)
    
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

    final_predictions = postprocess_qa_predictions(dataset, validation_features, raw_predictions.predictions)

    predictions = [v for k, v in final_predictions.items()]
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset]
    
    if save_dataframe_with_predictions:
        data = pd.DataFrame(dataset)   
        data['prediction_text'] = predictions
        data.to_json('./datasets/' + name + '.json', orient='records')

    return metric.compute(predictions=formatted_predictions, references=references)

### Evaluation of models on datasets



### SQuAD dataset 

In [12]:
# dataset from HuggingFace - they are the same
# datasets = load_dataset("squad")

In [13]:
# loading of corresponding dataset
data = pd.read_json('./datasets/valid_squad_with_predictions.json')

In [14]:
dataset = Dataset.from_pandas(data)

In [15]:
# the commented part of the code can be used for saving of squad dataset with predictions
# feel free to uncomment it, if you want to generate the dataset with predictions
metrics = model_evaluation_on_dataset(dataset) #, save_dataframe_with_predictions = True, name = m_name)
metrics

100%|██████████| 11/11 [00:43<00:00,  3.93s/ba]
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10784
  Batch size = 16


Post-processing 10570 example predictions split into 10784 features.


100%|██████████| 10570/10570 [00:25<00:00, 416.20it/s]


{'exact_match': 80.19867549668874, 'f1': 87.57572428424747}

### AdversarialQA dataset

In [16]:
# dataset from the HuggingFace - they are the same
# datasets = load_dataset("adversarial_qa", "adversarialQA")

In [17]:
# loading of corresponding dataset
data = pd.read_json('./datasets/adversarialqa_validation.json')

In [18]:
dataset = Dataset.from_pandas(data)

In [19]:
data = pd.DataFrame(dataset)

In [20]:
metrics = model_evaluation_on_dataset(dataset)
metrics

100%|██████████| 3/3 [00:11<00:00,  3.67s/ba]
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3010
  Batch size = 16


Post-processing 3000 example predictions split into 3010 features.


100%|██████████| 3000/3000 [00:06<00:00, 433.64it/s]


{'exact_match': 19.866666666666667, 'f1': 31.299247172832363}

### Natural Questions dataset

In [21]:
# loading of corresponding dataset
data = pd.read_json('./datasets/nq_dev_formatted.json')

In [22]:
dataset = Dataset.from_pandas(data)

In [23]:
metrics = model_evaluation_on_dataset(dataset)
metrics

100%|██████████| 3/3 [00:08<00:00,  2.76s/ba]
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2388
  Batch size = 16


Post-processing 2356 example predictions split into 2388 features.


100%|██████████| 2356/2356 [00:06<00:00, 384.33it/s]


{'exact_match': 55.730050933786075, 'f1': 67.82058427738443}

### TriviaQA dataset

In [24]:
# loading of corresponding dataset
data = pd.read_json('./datasets/triviaqa_dev_formatted.json')

In [25]:
dataset = Dataset.from_pandas(data)

In [26]:
metrics = model_evaluation_on_dataset(dataset)
metrics

100%|██████████| 10/10 [03:05<00:00, 18.57s/ba]
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36286
  Batch size = 16


Post-processing 9835 example predictions split into 36286 features.


100%|██████████| 9835/9835 [01:41<00:00, 97.34it/s] 


{'exact_match': 37.580071174377224, 'f1': 46.881850685798064}